In [90]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set_theme(style="darkgrid")

### Load the dataset

In [91]:
df = pd.read_excel('./data/employes_dataset.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  1000 non-null   int64         
 1   Nom                 1000 non-null   object        
 2   Prénom              1000 non-null   object        
 3   Âge                 1000 non-null   int64         
 4   Sexe                1000 non-null   object        
 5   Email               1000 non-null   object        
 6   Pays                1000 non-null   object        
 7   Ville               1000 non-null   object        
 8   Salaire (€)         996 non-null    float64       
 9   Date d'embauche     1000 non-null   datetime64[ns]
 10  Département         1000 non-null   object        
 11  Télétravail (%)     946 non-null    float64       
 12  Performance (Note)  952 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(2), objec

### Drop rows with missing values in the features columns

In [92]:
df = df.dropna(subset=[col for col in df.columns if col != 'Télétravail (%)'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  948 non-null    int64         
 1   Nom                 948 non-null    object        
 2   Prénom              948 non-null    object        
 3   Âge                 948 non-null    int64         
 4   Sexe                948 non-null    object        
 5   Email               948 non-null    object        
 6   Pays                948 non-null    object        
 7   Ville               948 non-null    object        
 8   Salaire (€)         948 non-null    float64       
 9   Date d'embauche     948 non-null    datetime64[ns]
 10  Département         948 non-null    object        
 11  Télétravail (%)     899 non-null    float64       
 12  Performance (Note)  948 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(7)
m

### Examine if `Télétravail (%)` is correlated with other variables


Create column `Ancienneté (années)` based on `Date d'embauche` column.


In [93]:
today = datetime.today()

df["Ancienneté (années)"] = (today - df["Date d'embauche"]).dt.days/365
df["Ancienneté (années)"] = df["Ancienneté (années)"].round(2)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   948 non-null    int64         
 1   Nom                  948 non-null    object        
 2   Prénom               948 non-null    object        
 3   Âge                  948 non-null    int64         
 4   Sexe                 948 non-null    object        
 5   Email                948 non-null    object        
 6   Pays                 948 non-null    object        
 7   Ville                948 non-null    object        
 8   Salaire (€)          948 non-null    float64       
 9   Date d'embauche      948 non-null    datetime64[ns]
 10  Département          948 non-null    object        
 11  Télétravail (%)      899 non-null    float64       
 12  Performance (Note)   948 non-null    float64       
 13  Ancienneté (années)  948 non-null    flo

Before I proceed to One-Hot-Encoding of `Pays`, `Département` and `Ville` columns.

In [94]:
# encoder = LabelEncoder()
# df['Département'] = encoder.fit_transform(df['Département'])
# df['Ville'] = encoder.fit_transform(df['Ville'])
# df['Télétravail (%)'] = encoder.fit_transform(df['Télétravail (%)'])

df = pd.get_dummies(df, columns=['Pays', 'Département', 'Ville'], drop_first=True)

In [95]:
correlation_matrix = df[[col for col in df.columns if df[col].dtype != 'object']].corr()["Télétravail (%)"].dropna().sort_values(ascending=False)
print(correlation_matrix)

Télétravail (%)             1.000000
Ville_West Matthew          0.080461
Ville_Brittanyview          0.080461
Département_Informatique    0.080049
Pays_Switzerland            0.075641
                              ...   
Pays_Panama                -0.068400
Âge                        -0.070233
Pays_Morocco               -0.075070
Pays_Spain                 -0.075070
Pays_Guatemala             -0.079849
Name: Télétravail (%), Length: 1121, dtype: float64


### I don't see any strong correlation between `Télétravail (%)` and other columns but I will continue with `Pays`, `Département` and `Ville` columns as features.

In [96]:
# Select columns who match the pattern Pays*, Département* and Ville*
print(df.shape)

df = df.loc[:,df.columns.str.match(r"^(Pays|Département|Ville)_\w+$") | df.columns.isin(["Télétravail (%)", "Ancienneté (années)"])]

print(df.shape)

(948, 1168)
(948, 641)


In [97]:
df.isnull().sum()

Télétravail (%)        49
Ancienneté (années)     0
Pays_Albania            0
Pays_Algeria            0
Pays_Andorra            0
                       ..
Ville_Wrightport        0
Ville_Wyattborough      0
Ville_Yangstad          0
Ville_Yoderberg         0
Ville_Zacharyport       0
Length: 641, dtype: int64

In [98]:
df_with_empty_line = df[df["Télétravail (%)"].isna()]
df_without_empty_line = df[df["Télétravail (%)"].notna()]

In [99]:
x_train, x_test , y_train, y_test = train_test_split(df_without_empty_line.drop(columns=["Télétravail (%)"]), df_without_empty_line["Télétravail (%)"],test_size=0.2, random_state=42)

In [100]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)

x_test = scaler.transform(x_test)

In [101]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [102]:
y_pred = model.predict(x_test)

r2 = r2_score(y_test, y_pred)
print(r2)

mse = mean_squared_error(y_test, y_pred)
print(mse)

-0.384841802317905
1524.5398562282705


# Let's predict the `Télétravail (%)` for the employees who have not filled it yet.

In [103]:
df_with_empty_line["Télétravail (%)"] = model.predict(scaler.transform(df_with_empty_line.drop(columns=["Télétravail (%)"])))

df_with_empty_line[["Télétravail (%)"]]

/tmp/ipykernel_54334/2209509878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_empty_line["Télétravail (%)"] = model.predict(scaler.transform(df_with_empty_line.drop(columns=["Télétravail (%)"])))


,Télétravail (%)
7,6.358769
25,86.942018
55,81.292505
102,-0.649120
121,78.151403
142,50.923393
145,35.974548
150,75.653297
165,63.607806
218,13.196820
